In [3]:
import os
import time
import threading

#### Current Thread

In [ ]:
print('Program Enter')
print(f'Process id - {os.getpid()}')
print(f'Current Thread name - {threading.current_thread().name}')
print('Sleeping...')
time.sleep(5)
print('Wake up...')
threading.current_thread().name = 'MyThread'
print(f'Current Thread name - {threading.current_thread().name}')
print('Program Exit')

#### Create a new Thread
- Create an instance of Thread class and set a target function for it to run
- Write a sub-class of the Thread class, override the run() method, create instance of the sub-class

In [55]:
def run():
    for i in range(1,6):
        print(f'{i} --> Thread name -> {threading.current_thread().name}, Process id -> {os.getpid()}')
        time.sleep(0.1)

In [56]:
print(f'Main Thread Enter, Process id -> {os.getpid()}')

t1 = threading.Thread(target=run)
t1.name='FirstThread'
t1.start()

print(f't1 is alive -> {t1.is_alive()}')

t1.join() # current thread (main thread) goes to wait state, gets notified when t1 is dead 

print(f't1 is alive -> {t1.is_alive()}')  # Not alive

print('Main Thread Exit')

Main Thread Enter, Process id -> 16646
1 --> Thread name -> FirstThread, Process id -> 16646
t1 is alive -> True
2 --> Thread name -> FirstThread, Process id -> 16646
3 --> Thread name -> FirstThread, Process id -> 16646
4 --> Thread name -> FirstThread, Process id -> 16646
5 --> Thread name -> FirstThread, Process id -> 16646
t1 is alive -> False
Main Thread Exit


In [57]:
print(f'Main Thread Enter, Process id -> {os.getpid()}')

t1 = threading.Thread(target=run, name='Thread-01')
t2 = threading.Thread(target=run, name='Thread-02')

t1.start()
t2.start()

t1.join() # current thread (main thread) goes to wait state, gets notified when t1 is dead 
t2.join()

print('Main Thread Exit')

Main Thread Enter, Process id -> 16646
1 --> Thread name -> Thread-01, Process id -> 16646
1 --> Thread name -> Thread-02, Process id -> 16646
2 --> Thread name -> Thread-01, Process id -> 16646
2 --> Thread name -> Thread-02, Process id -> 16646
3 --> Thread name -> Thread-01, Process id -> 16646
3 --> Thread name -> Thread-02, Process id -> 16646
4 --> Thread name -> Thread-01, Process id -> 16646
4 --> Thread name -> Thread-02, Process id -> 16646
5 --> Thread name -> Thread-01, Process id -> 16646
5 --> Thread name -> Thread-02, Process id -> 16646
Main Thread Exit


#### Create a Thread whose target function accepts numbers (no limit), adds them and display the sum 

In [ ]:
def add(*args):
    print(f'Thread name -> {threading.current_thread().name}, Sum -> {sum(args)}')

#-----------------------------------

t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t2 = threading.Thread(target=add, name='Second', args=(11,22,33))

t1.start()
t2.start()

#### Create a Thread by Inheritence
1. Write a sub-class of the Thread class
2. Default target method for such threads is the 'run()' method, hence we override this method
3. In case the target method is any other method, we explicitly set the attribute (t.run)
4. Arguments are passed through the attribute t.args and accessed using self.args

In [ ]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} --> Thread name -> {threading.current_thread().name}, Process id -> {os.getpid()}')
            os.sched_yield()  # Running --> Ready  (Give up the CPU)
            #time.sleep(0)    # Running --> Sleep --> Ready

In [ ]:
t1 = MyThread()
t1.name = 'First-Thread'
t1.run = t1.do_something  # (in case the method is not run)
t1.start()

#### Write a sub-class of Thread, name it as AddThread
- The target method would sum the args and display the same

In [ ]:
class AddThread(threading.Thread):
    def run(self):
        print(f'{threading.current_thread().name} --> Sum: {sum(self.args)}')

In [ ]:
t1 = AddThread()
t1.name = 'Thread-001'
t1.args = (1,2,3,4,5)

t2 = AddThread()
t2.name = 'Thread-002'
t2.args = (10,20,30)

t1.start()
t2.start()

#### Daemon threads
- These are the threads which run the background
- Purpose of the daemon threads is to give service to the application
- If daemon threads are the only threads running, then the Runtime would quit
- By default a thread is not daemon unless we set it as daemon thread (t.daemon = True)

In [ ]:
import time
import threading

def daemon_task():
    while True:
        print('\tThis is Daemon')
        time.sleep(1)

t1 = threading.Thread(target=daemon_task, daemon=True)
t1.start()

for i in range(1,11):
    print(f'{i} --> This is Main')
    time.sleep(1)

In [2]:
!python daemon_demo.py

	This is Daemon
1 --> This is Main
	This is Daemon
2 --> This is Main
3 --> This is Main
	This is Daemon
4 --> This is Main
	This is Daemon
5 --> This is Main
	This is Daemon
6 --> This is Main
	This is Daemon
7 --> This is Main
	This is Daemon
8 --> This is Main
	This is Daemon
9 --> This is Main
	This is Daemon
10 --> This is Main
	This is Daemon


#### Thread assignment
- Write a class named Resource
- It will have an instance field named 'data', initilize it to 0 
- It will have an instance method named 'do_something'
- do_something() method will increment data by 1, then display the name of the curent thread and current value of the data 
- The resource class method 'do_something' will be the target method for a Thread

#### Non Thread-safe Resource (can result in Race condition)

In [41]:
class Resource:
    def __init__(self):
        self.data = 0
        
    def do_something(self):
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')    

#### Thread-safe Resource (safely used in Multi Threaded environment)

In [48]:
class Resource:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
        
    def do_something(self):
        self.lock.acquire()
        self.data += 1
        for i in range(1000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')
        self.lock.release()

In [50]:
class Resource2:
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
        
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(1000000):
                pass
            print(f'{threading.current_thread().name} --> {self.data}')

In [52]:
r = Resource2()

for i in range(1,6):
    t1 = threading.Thread(target = r.do_something, name=f'Thread-0{i}')
    t1.start()

Thread-01 --> 1
Thread-02 --> 2
Thread-03 --> 3
Thread-04 --> 4
Thread-05 --> 5


#### MultiProcessing
- MultiThreading --> Concurrent processing (uses only 1 CPU which is shared by all the Threads) 
- MultiProcessing --> Parallel processing (uses as many CPUs as available, each task runs in a separate process) 

In [58]:
import os
import multiprocessing

In [59]:
print('Available Processors: ', os.cpu_count())

Available Processors:  8


In [65]:
def worker1():
    print(f'Worker-1, Process id: {os.getpid()}')

def worker2():
    print(f'Worker-2, Process id: {os.getpid()}')
#------------------------------------------------------

print(f'Main Process id: {os.getpid()}')    

p1 = multiprocessing.Process(target=worker1)
p2 = multiprocessing.Process(target=worker2)

p1.start()
p2.start()
    

print('\nBefore Join')
print('Worker-1 is alive:', p1.is_alive())    
print('Worker-2 is alive:', p2.is_alive())    

p1.join()
p2.join()
    
print('\nAfter Join')
print('Worker-1 is alive:', p1.is_alive())    
print('Worker-2 is alive:', p2.is_alive())        
    
print('\nMain Process exit')

Main Process id: 16646
Worker-1, Process id: 43018
Worker-2, Process id: 43021

Before Join
Worker-1 is alive: True
Worker-2 is alive: True

After Join
Worker-1 is alive: False
Worker-2 is alive: False

Main Process exit


#### Create 2 Process objects
- One will square a given number and display the result (given as a parameter)
- Other will cube a given number and display the result (given as a parameter)
- Parameters are passed as args (similar to args concept in threading)
- Make sure that only one Process prints at any given time

In [71]:
import os
import multiprocessing

def square(lock,num):
    lock.acquire()
    print(f'Square Process id: {os.getpid()}')
    print(f'Square of {num} is {num ** 2}')
    lock.release()

def cube(lock,num):
    with lock:
        print(f'Cube Process id: {os.getpid()}')
        print(f'Cube of {num} is {num ** 3}')

if __name__ == '__main__':   
    print(f'Main Process id: {os.getpid()}')

    lock = multiprocessing.Lock()
    
    p1 = multiprocessing.Process(target=square, args=(lock,10))
    p2 = multiprocessing.Process(target=cube, args=(lock,10))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print('Main Process exit')

Main Process id: 16646
Square Process id: 47657
Square of 10 is 100
Cube Process id: 47658
Cube of 10 is 1000
Main Process exit


#### Sharing data in MultiProcessing through Queue

In [76]:
from multiprocessing import Process, Queue

def func(queue):
    value = queue.get()
    print(f'In Child Process, got: {value}')
    for i in range(1,value+1):
        queue.put(i ** 2)
    
#----------------------------

if __name__ == '__main__':   
    queue = Queue()
    p = Process(target=func, args=(queue,))
    p.start()
    queue.put(5)
    p.join()
    
    print('In Parent Process')
    while not queue.empty():
        print(queue.get())

In Child Process, got: 5
In Parent Process
1
4
9
16
25


#### Sharing data in MultiProcessing through Pipe

In [75]:
from multiprocessing import Process, Pipe

def func(child_conn):
    value = child_conn.recv()
    print(f'In Child Process, got: {value}')
    for i in range(1,value+1):
        child_conn.send(i ** 2)
#----------------------------

if __name__ == '__main__':   
    parent_conn, child_conn = Pipe()
    p = Process(target=func, args=(child_conn,))
    p.start()
    num = 5
    parent_conn.send(num)
    print('In Parent Process')
    for i in range(num):
        print(parent_conn.recv())
    
    p.join()
    
    parent_conn.close() # No further send or recv is possible
    print('Pipe closed')

In Child Process, got: 5
In Parent Process
1
4
9
16
25
Pipe closed
